### CUSTOM METRICS

+ Losses and Metrics are completely different concepts'
+ losses are used by gradient descent to train the model.
+ metrics are used to evaluate the model.
+ defining the custom metrics is same as the custom loss function.
+ we can use the huber_loss function as well.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
## fetch the data.
housing = fetch_california_housing()
# housing.data
# housing.target
print(housing.keys())
print()
print("====================================Complete Dataset======================================")
## X_train_full, X_test, y_train_full, y_test
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1,1), random_state = 42)
print(X_train_full.shape)
print(X_test.shape)
print(y_train_full.shape)
print(y_test.shape)
print()
print("====================================Dataset After Splitting to Validation Set======================================")
## X_valid, X_test, y_valid, y_test
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)
print()

print("=================================Standardize the Data==============================================")
## Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print(X_train_scaled.shape)
X_valid_scaled = scaler.transform(X_valid)
print(X_valid_scaled.shape)
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled.shape)

## defining the input shape
input_shape = X_train.shape[1:]
print(input_shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

====================================Complete Dataset======================================
(15480, 8)
(5160, 8)
(15480, 1)
(5160, 1)

====================================Dataset After Splitting to Validation Set======================================
(11610, 8)
(3870, 8)
(11610, 1)
(3870, 1)

=================================Standardize the Data==============================================
(11610, 8)
(3870, 8)
(5160, 8)
(8,)


In [6]:
## defining the huber function
## this is the loss function
def huber_fn(y_true, y_pred):
    ## define the error from y_true and y_pred, that will be the difference
    error = y_true - y_pred
    #print(error)
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn


In [7]:
## model development
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

## comoile the model
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])
## ft the model
model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 1.7428 - huber_fn: 0.7451
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.5565 - huber_fn: 0.2502


In [9]:
## using the huber_function as the loss.
## compile the model
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=[create_huber(2.0)])
## fit the model
sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled, y_train, epochs=2, sample_weight=sample_weight)

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.1174 - huber_fn: 0.2317
Epoch 2/2
363/363 [==============================] - 1s 1ms/step - loss: 0.1122 - huber_fn: 0.2218


In [10]:
history.history["loss"][0], history.history["huber_fn"][0] * sample_weight.mean()

(0.11742295324802399, 0.11525372907047944)

STREAMING METRICS

In [31]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])
print(precision.result())
print(precision.variables)
print(precision.reset_states())

## creating the streaming metrics
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}
    
m = HuberMetric(2.)

# total = 2 * |10 - 2| - 2²/2 = 14
# count = 1
# result = 14 / 1 = 14
print(m(tf.constant([[2.]]), tf.constant([[10.]])) )

# total = total + (|1 - 0|² / 2) + (2 * |9.25 - 5| - 2² / 2) = 14 + 7 = 21
# count = count + 2 = 3
# result = total / count = 21 / 3 = 7
m(tf.constant([[0.], [5.]]), tf.constant([[1.], [9.25]]))

print(m.result())

print(m.variables)
print(m.reset_states())
print(m.variables)



tf.Tensor(0.5, shape=(), dtype=float32)
[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>, <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]
None
tf.Tensor(14.0, shape=(), dtype=float32)
tf.Tensor(7.0, shape=(), dtype=float32)
[<tf.Variable 'total:0' shape=() dtype=float32, numpy=21.0>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>]
None
[<tf.Variable 'total:0' shape=() dtype=float32, numpy=0.0>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=0.0>]


In [35]:
## model development
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=[HuberMetric(2.0)])

model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32), epochs=2)

model.save("my_model_with_a_custom_metric.h5")

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.9785 - huber_metric_9: 0.9785
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2564 - huber_metric_9: 0.2564


In [37]:
## when loading the model again
## we need to specify the metrics again
model = keras.models.load_model("my_model_with_a_custom_metric.h5",
                                custom_objects={"huber_fn": create_huber(2.0),
                                                "HuberMetric": HuberMetric})

model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.2332 - huber_metric_9: 0.2332
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2248 - huber_metric_9: 0.2248


In [38]:
model.metrics[-1].threshold

2.0

In [41]:
## simple way to create the class
class HuberMetric(keras.metrics.Mean):
    def __init__(self, threshold=1.0, name='HuberMetric', dtype=None):
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        super().__init__(name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetric, self).update_state(metric, sample_weight)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}  
    
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=keras.losses.Huber(2.0), optimizer="nadam", weighted_metrics=[HuberMetric(2.0)])

sample_weight = np.random.rand(len(y_train))
history = model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32),
                    epochs=2, sample_weight=sample_weight)

history.history["loss"][0], history.history["HuberMetric"][0] * sample_weight.mean()

model.save("my_model_with_a_custom_metric_v2.h5")

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.4583 - HuberMetric: 0.9167
Epoch 2/2
363/363 [==============================] - 1s 1ms/step - loss: 0.1293 - HuberMetric: 0.2586


In [42]:
## when loading the mdoel again
## we need to specify the parameters again.
model = keras.models.load_model("my_model_with_a_custom_metric_v2.h5",
                                custom_objects={"HuberMetric": HuberMetric})
model.fit(X_train_scaled.astype(np.float32), y_train.astype(np.float32), epochs=2)

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.2351 - HuberMetric: 0.2351
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2258 - HuberMetric: 0.2258


In [43]:
model.metrics[-1].threshold

2.0